In [1]:
#Better
!pip install requests BeautifulSoup4 fire

     |████████████████████████████████| 81kB 3.2MB/s 
  Created wheel for fire: filename=fire-0.3.1-py2.py3-none-any.whl size=111005 sha256=3f1ef0b60b949773083773225e6dea01fa1c9d3b978e838313184aeea6a8dd0a
  Stored in directory: /root/.cache/pip/wheels/c1/61/df/768b03527bf006b546dce284eb4249b185669e65afc5fbb2ac
Successfully built fire


In [3]:
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup
import pandas as pd
import os, sys

import fire

In [4]:
#%%writefile ../pyscrap_url.py

def simple_get(url):
    """
    Attempts to get the content at `url` by making an HTTP GET request.
    If the content-type of response is some kind of HTML/XML, return the
    text content, otherwise return None.
    """
    try:
        with closing(get(url, stream=True)) as resp:
            if is_good_response(resp):
                return resp.content  #.encode(BeautifulSoup.original_encoding)
            else:
                return None

    except RequestException as e:
        log_error('Error during requests to {0} : {1}'.format(url, str(e)))
        return None


def is_good_response(resp):
    """
    Returns True if the response seems to be HTML, False otherwise.
    """
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200 
            and content_type is not None 
            and content_type.find('html') > -1)


def log_error(e):
    """
    It is always a good idea to log errors. 
    This function just prints them, but you can
    make it do anything.
    """
    print(e)
    
def get_elements(url, tag='',search={}, fname=None):
    """
    Downloads a page specified by the url parameter
    and returns a list of strings, one per tag element
    """
    
    if isinstance(url,str):
        response = simple_get(url)
    else:
        #if already it is a loaded html page
        response = url

    if response is not None:
        html = BeautifulSoup(response, 'html.parser')
        
        res = []
        if tag:    
            for li in html.select(tag):
                for name in li.text.split('\n'):
                    if len(name) > 0:
                        res.append(name.strip())
                       
                
        if search:
            soup = html            
            
            
            r = ''
            if 'find' in search.keys():
                print('findaing',search['find'])
                soup = soup.find(**search['find'])
                r = soup

                
            if 'find_all' in search.keys():
                print('findaing all of',search['find_all'])
                r = soup.find_all(**search['find_all'])
   
            if r:
                for x in list(r):
                    if len(x) > 0:
                        res.extend(x)
            
        return res

    # Raise an exception if we failed to get any data from the url
    raise Exception('Error retrieving contents at {}'.format(url))    
    
def get_element(url, tag='',search={}, fname=None):
    """
    Downloads a page specified by the url parameter
    and returns a list of strings, one per tag element
    """
    if isinstance(url,str):
        response = simple_get(url)
    else:
        #if already it is a loaded html page
        response = url
    if response is not None:
        html = BeautifulSoup(response, 'html.parser')
        res = []
        if tag:    
            for li in html.select(tag):
                for name in li.text.split('\n'):
                    if len(name) > 0:
                        res.append(name.strip())
        if search:
            soup = html            
            r = ''
            if 'find' in search.keys():
                print('findaing',search['find'])
                soup = soup.find(**search['find'])
                r = soup
            if 'find_all' in search.keys():
                print('findaing all of',search['find_all'])
                r = soup.find_all(**search['find_all'])
            if r:
                for x in list(r):
                    if len(x) > 0:
                        res.extend(x)
        return res
    # Raise an exception if we failed to get any data from the url
    raise Exception('Error retrieving contents at {}'.format(url))    
    
    
if get_ipython().__class__.__name__ == '__main__':
    fire(get_tag_elements)

##### Web scrape 100 most influential Twitter users in Africa using Python or Bash to obtain the 100 African twitter influencers. This website uses three key metrics to find the top 100 influencers a. Popularity (Retweet Influence): measured by the number of Retweets and Likes users get b. Reach (Indegree Influence): measured by the size of their audience c. Relevance (Mentions Influence): measured by the relevancy of their content 


In [5]:
res = get_elements('https://africafreak.com/100-most-influential-twitter-users-in-africa',tag='h2')
res

['100. Jeffrey Gettleman (@gettleman)',
 '99. Africa24 Media (@a24media)',
 '98. Scapegoat (@andiMakinana)',
 '97. Africa Check (@AfricaCheck)',
 '96. James Copnall (@JamesCopnall)',
 '95. Online Africa (@oafrica)',
 '94. Patrick Ngowi (@PatrickNgowi)',
 '93. DOS African Affairs (@StateAfrica)',
 '92. MoadowAJE (@Moadow)',
 '91. Brendan Boyle (@BrendanSAfrica)',
 '90. City of Tshwane (@CityTshwane)',
 '89. VISI Magazine (@VISI_Mag)',
 '88. andBeyond (@andBeyondSafari)',
 '87. This Is Africa (@ThisIsAfricaTIA)',
 '86. Sarah Carter (@sarzss)',
 '85. The EIU Africa team (@TheEIU_Africa)',
 '84. Investing In Africa (@InvestInAfrica)',
 '83. Barry Malone (@malonebarry)',
 '82. ARTsouthAFRICA (@artsouthafrica)',
 '81. Kahn Morbee (@KahnMorbee)',
 '80. Jamal Osman (@JamalMOsman)',
 '79. iamsuede™ (@iamsuede)',
 '78. Mike Stopforth (@mikestopforth)',
 '77. Equal Education (@equal_education)',
 '76. Tristan McConnell (@t_mcconnell)',
 '75. Kate Forbes (@forbeesta)',
 '74. Vanessa Raphaely (@hur

##### I extracted the twitter handles for the 100 Non-Government Influentials.

In [7]:
non_govt_influencers = res
non_govt_influencers[:5]
non_govt_influencers = pd.Series(non_govt_influencers)
user_twitter_handle = [i.split('(')[-1].strip(')') for i in non_govt_influencers]

for i in user_twitter_handle[100:]:
    user_twitter_handle.remove(i)
user_twitter_handle

['@gettleman',
 '@a24media',
 '@andiMakinana',
 '@AfricaCheck',
 '@JamesCopnall',
 '@oafrica',
 '@PatrickNgowi',
 '@StateAfrica',
 '@Moadow',
 '@BrendanSAfrica',
 '@CityTshwane',
 '@VISI_Mag',
 '@andBeyondSafari',
 '@ThisIsAfricaTIA',
 '@sarzss',
 '@TheEIU_Africa',
 '@InvestInAfrica',
 '@malonebarry',
 '@artsouthafrica',
 '@KahnMorbee',
 '@JamalMOsman',
 '@iamsuede',
 '@mikestopforth',
 '@equal_education',
 '@t_mcconnell',
 '@forbeesta',
 '@hurricanevaness',
 '@BBCKarenAllen',
 '@jaxpanik',
 '@thisisafrica',
 '@audisouthafrica',
 '@ONEinAfrica',
 '@Hamza_Africa',
 '@drewfhinshaw',
 '@africatechie',
 '@cx73',
 '@ayittey',
 '@MercedesBenz_SA',
 '@africagathering',
 '@okayafrica',
 '@mary_harper',
 '@savetherhino',
 '@africamedia_CPJ',
 '@TechCentral',
 '@GautengProvince',
 '@Aynte',
 '@daniel_howden',
 '@rangerdiaries',
 '@TheStar_news',
 '@schneiderhome',
 '@Afrinnovator',
 '@theafricareport',
 '@CityofJoburgZA',
 '@ThinkAfricaFeed',
 '@AfricaGoodNews',
 '@willintune',
 '@cnbcafrica',
 

##### I queried twitter to get the tweets of the Non-Government Influentials gotten above.

In [56]:
import tweepy
consumer_key = "cd6xXCBVFRKohny14KUvgBcyf"
consumer_secret = "FvHm7Cd2g73FdZ269VjbZSE3ymngOse3p8RXfpavwbmovupt8d"
access_token = "1246485711282462722-8JMUgu4vunrqXs04hfASXgZ1rhP1f3"
access_token_secret = "aLi3JSDTZ79XDutDFXiSVU9kqrm88TKNUZF9bG7pIskoB"

# Creating the authentication object
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
# Setting your access token and secret
auth.set_access_token(access_token, access_token_secret)
# Creating the API object while passing in auth information
api = tweepy.API(auth) 
Tweets_NonGov_Influentials = []
    
for x in range(0, len(user_twitter_handle)):

    # The Twitter user who we want to get tweets from
    name = user_twitter_handle[x]
    try:
      # Calling the user_timeline function with our parameters
      results = api.user_timeline(id=name, count=10)

      # foreach through all tweets pulled
      for tweet in results:
      # printing the text stored inside the tweet object
          Tweets_NonGov_Influentials.append(tweet.text)
    except tweepy.TweepError as e:
        continue

from pandas import DataFrame
df_tweets_nongovInfluencers = DataFrame(Tweets_NonGov_Influentials, columns=['Tweets of Influentials'])
df_tweets_nongovInfluencers

,Tweets of Influentials
0,What an inspiring interchange. Way to go Luke!...
1,"It's a fantastic school. Congrats, farmer's so..."
2,"RT @ndtv: 99 Doctors Died Due To COVID-19, Say..."
3,RT @shaikubaid: Specter of Caste in Silicon Va...
4,RT @SavvyPriya: @gettleman @HariNYT It's a ver...
...,...
892,RT @franklinleonard: “I have rape-colored skin...
893,There are people fighting to walk around a sup...
894,RT @SawyerHackett: Never would have thought a ...
895,@kimlatricejones @DEADLINE 🙌🏽


##### I saved the tweets of the Non-government Influentials gotten above to a dataframe

#####  By analysing their followers count, number of likes, number of retweets, number of people they follow, number of tweets they shared, score each of the influencers and top government officials as follows: 
a. popularity_score = #retweets + #likes 

b. reach_score = #followers - #they follow 

c. relevance_score = #comments + #mentions in other tweets 


##### Analysing for Non-Government Influencers to get the followers count, likes count, following count, tweets count for each user

In [12]:
screenname_NonGov_Influentials = []
no_ofLikes_NonGov_Influentials = []
no_ofFollowing_NonGov_Influentials = []
no_ofTweetsCount_NonGov_Influentials = []
no_ofFollowers_NonGov_Influentials = []

for x in range(0, len(user_twitter_handle)):
    
    # The Twitter user who we want to get followers count for
    name = user_twitter_handle[x]
    try:
        # Calling the user_timeline function with our parameters
        results = api.get_user(id=name)
        screenname_NonGov_Influentials.append(results.screen_name)
        no_ofFollowers_NonGov_Influentials.append(results.followers_count)
        no_ofLikes_NonGov_Influentials.append(results.favourites_count)
        no_ofFollowing_NonGov_Influentials.append(results.friends_count)
        no_ofTweetsCount_NonGov_Influentials.append(results.statuses_count)
    except tweepy.TweepError as e:
        continue

from pandas import DataFrame
df_scrrenname_nongovInfluencers = DataFrame(screenname_NonGov_Influentials,columns=['Screen Name of Non-Government Influentials'])
df_followerscount_nongovInfluencers = DataFrame(no_ofFollowers_NonGov_Influentials,columns=['No Of Followers of Non-Government Influentials'])
df_likescount_nongovInfluencers = DataFrame(no_ofLikes_NonGov_Influentials,columns=['No Of Likes of Non-Government Influentials'])
df_followingcount_nongovInfluencers = DataFrame(no_ofFollowing_NonGov_Influentials,columns=['No Of Following for Non-Government Influentials'])
df_tweetscount_nongovInfluencers = DataFrame(no_ofTweetsCount_NonGov_Influentials,columns=['No Of Tweets Shared for Non-Government Influentials'])

concatenated_list = pd.concat([df_scrrenname_nongovInfluencers, df_followerscount_nongovInfluencers,df_likescount_nongovInfluencers, df_followingcount_nongovInfluencers, df_tweetscount_nongovInfluencers], axis=1)
concatenated_list

,Screen Name of Non-Government Influentials,No Of Followers of Non-Government Influentials,No Of Likes of Non-Government Influentials,No Of Following for Non-Government Influentials,No Of Tweets Shared for Non-Government Influentials
0,gettleman,25704,9,37,3770
1,a24media,31291,1621,3059,16869
2,AndiMakinana,101277,9034,2837,142227
3,AfricaCheck,68078,3916,4592,27323
4,JamesCopnall,21966,3625,5050,19439
...,...,...,...,...,...
88,Julius_S_Malema,3128917,4647,652,37210
89,News24,3577449,1143,631,322845
90,SAPresident,18,65,14,19
91,GarethCliff,1974657,34001,356,31626


##### To get number of retweets, popularity scores, reach scores and relevance scores for each user that is a Non-Gov Influencer

In [41]:
no_ofRetweets_NonGov_Influentials = []     
popularityscores_NonGov_influencers = []
reachscores_NonGov_influencers = []

#get number of retweets
for x in range(0, len(user_twitter_handle)):
    name = user_twitter_handle[x]
    try:
      Retweet_count = 0
      for status in tweepy.Cursor(api.user_timeline,id=name).items():
        for 
          Retweet_count += status.retweet_count
      no_ofRetweets_NonGov_Influentials.append(Retweet_count)
    except tweepy.TweepError as e:
        continue

#get popularity scores for each user
for i in range(0, len(no_ofRetweets_NonGov_Influentials)):
    popularity_score_nongov = no_ofRetweets_NonGov_Influentials[i] + no_ofLikes_NonGov_Influentials[i]
    popularityscores_NonGov_influencers.append(popularity_score_nongov)

#get reach scores for each user
for i in range(0, len(no_ofFollowers_NonGov_Influentials)):
    reachscore = no_ofFollowers_NonGov_Influentials[i] - no_ofFollowing_NonGov_Influentials[i]
    reachscores_NonGov_influencers.append(reachscore)

from pandas import DataFrame
df_retweetcount_nongovInfluencers = DataFrame (no_ofRetweets_NonGov_Influentials,columns=['No Of Retweets of Non-Government Influentials'])
df_popularityscores_nongovInfluencers = DataFrame (popularityscores_NonGov_influencers,columns=['Popularity Scores for Non-Government Officials'])
df_reachscores_nongov_Influencers = DataFrame (reachscores_NonGov_influencers,columns=['Reach Scores for Non-Government Influencers'])

concatenated_list = pd.concat([df_scrrenname_nongovInfluencers, df_retweetcount_nongovInfluencers,df_popularityscores_nongovInfluencers,df_reachscores_nongov_Influencers], axis=1)
concatenated_list

,Screen Name of Non-Government Influentials,No Of Retweets of Non-Government Influentials,Popularity Scores for Non-Government Officials,Reach Scores for Non-Government Influencers
0,gettleman,NaN,NaN,25667
1,a24media,NaN,NaN,28232
2,AndiMakinana,NaN,NaN,98440
3,AfricaCheck,NaN,NaN,63486
4,JamesCopnall,NaN,NaN,16916
...,...,...,...,...
88,Julius_S_Malema,NaN,NaN,3128265
89,News24,NaN,NaN,3576818
90,SAPresident,NaN,NaN,4
91,GarethCliff,NaN,NaN,1974301


### Webscraped the Government officials

In [15]:
url= 'https://www.atlanticcouncil.org/blogs/africasource/african-leaders-respond-to-coronavirus-on-twitter/#east-africa'
response = get(url).content
res_gov = get_elements(response, tag='blockquote')
gov_officials_twitter_handles = []
for r in res_gov:
    split_data = r.split('— ',maxsplit=1)[1].rsplit('(',maxsplit=1)
    name = split_data[0].split(',')[0].strip()
    handle =  split_data[1].rsplit(')',maxsplit=1)[0]
    gov_officials_twitter_handles.append(handle)
    print(f'{name} : {handle}')

Eswatini Government : @EswatiniGovern1
Malawi Government : @MalawiGovt
Hage G. Geingob : @hagegeingob
Seychelles Ministry of Finance : @FinanceSC
PresidencyZA : @PresidencyZA
Ministry of Health Zambia : @mohzambia
President of Zimbabwe : @edmnangagwa
MinSantédj : @MinSantedj
Yemane G. Meskel : @hawelti
State House Kenya : @StateHouseKenya
Paul Kagame : @PaulKagame
Mohamed Farmaajo : @M_Farmaajo
H.E Hussein Abdelbagi Akol : @SouthSudanGov
Abdalla Hamdok : @SudanPMHamdok
TanzaniaSpokesperson : @TZSpokesperson
Yoweri K Museveni : @KagutaMuseveni
MOFA/MRE -(Angola) : @angola_Mirex
Amb. Willy Nyamitwe : @willynyamitwe
Chérif Mahamat Zene : @Cherif_MZ
Présidence RDC 🇨🇩 : @Presidence_RDC
Ali Bongo Ondimba : @PresidentABO
Présidence du Bénin : @PresidenceBenin
Roch KABORE : @rochkaborepf
Presidente Cabo Verde : @PresidenciaCV
Alassane Ouattara : @AOuattara_PRCI
State House of The Gambia : @Presidency_GMB
Nana Akufo-Addo : @NAkufoAddo
Pr. Alpha CONDÉ : @President_GN
Umaro Sissoco Embalo : @USEm

Extract the government officials handles only

In [16]:
gov_officials_twitter_handles

['@EswatiniGovern1',
 '@MalawiGovt',
 '@hagegeingob',
 '@FinanceSC',
 '@PresidencyZA',
 '@mohzambia',
 '@edmnangagwa',
 '@MinSantedj',
 '@hawelti',
 '@StateHouseKenya',
 '@PaulKagame',
 '@M_Farmaajo',
 '@SouthSudanGov',
 '@SudanPMHamdok',
 '@TZSpokesperson',
 '@KagutaMuseveni',
 '@angola_Mirex',
 '@willynyamitwe',
 '@Cherif_MZ',
 '@Presidence_RDC',
 '@PresidentABO',
 '@PresidenceBenin',
 '@rochkaborepf',
 '@PresidenciaCV',
 '@AOuattara_PRCI',
 '@Presidency_GMB',
 '@NAkufoAddo',
 '@President_GN',
 '@USEmbalo',
 '@PresidenceMali',
 '@CheikhGhazouani',
 '@IssoufouMhm',
 '@MBuhari',
 '@Macky_Sall',
 '@PresidentBio',
 '@MSPS_Togo']

In [51]:
import tweepy
consumer_key = "cd6xXCBVFRKohny14KUvgBcyf"
consumer_secret = "FvHm7Cd2g73FdZ269VjbZSE3ymngOse3p8RXfpavwbmovupt8d"
access_token = "1246485711282462722-8JMUgu4vunrqXs04hfASXgZ1rhP1f3"
access_token_secret = "aLi3JSDTZ79XDutDFXiSVU9kqrm88TKNUZF9bG7pIskoB"

# Creating the authentication object
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
# Setting your access token and secret
auth.set_access_token(access_token, access_token_secret)
# Creating the API object while passing in auth information
api = tweepy.API(auth) 
Tweets_Gov_Officials = []
    
for x in range(0, len(gov_officials_twitter_handles)):

    # The Twitter user who we want to get tweets from
    name = gov_officials_twitter_handles[x]

    # Calling the user_timeline function with our parameters
    results = api.user_timeline(id=name)

    # foreach through all tweets pulled
    for tweet in results:
    # printing the text stored inside the tweet object
        Tweets_Gov_Officials.append(tweet.text)

from pandas import DataFrame
df_tweets = DataFrame (Tweets_Gov_Officials,columns=['Tweets of Government Officials'])
df_tweets

,Tweets of Government Officials
0,Ministerial Statement: Minister of Health Lizz...
1,Ministerial Statement: Minister of Health Lizz...
2,Minister of Health Lizzie Nkosi announces one ...
3,Ministerial Statement: Minister of Health Lizz...
4,Ambassador Aragones: “When this project is imp...
...,...
715,3è édition campagne de distribution des Mousti...
716,10 èmes journées de l’ordre national des pharm...
717,Une campagne de distribution de moustiquaires ...
718,Le 1er Mai 2017 célébrée par les agents du min...


##### Analysing for Government Influencers to get the followers count for each user

In [37]:
screenname_Gov_Influentials = []
no_ofLikes_Gov_Influentials = []
no_ofFollowing_Gov_Influentials = []
no_ofTweetsCount_Gov_Influentials = []
no_ofFollowers_Gov_Influentials = []
no_ofRetweets_Gov_Influentials = []     
popularityscores_Gov_Influentials = []
reachscores_Gov_Influentials = []

for x in range(0, len(gov_officials_twitter_handles)):
    
    # The Twitter user who we want to get followers count for
    name = gov_officials_twitter_handles[x]
    try:
        # Calling the user_timeline function with our parameters
        results = api.get_user(id=name)
        screenname_Gov_Influentials.append(results.screen_name)
        no_ofFollowers_Gov_Influentials.append(results.followers_count)
        no_ofLikes_Gov_Influentials.append(results.favourites_count)
        no_ofFollowing_Gov_Influentials.append(results.friends_count)
        no_ofTweetsCount_Gov_Influentials.append(results.statuses_count)
    except tweepy.TweepError as e:
        continue

from pandas import DataFrame
df_scrrenname_govInfluencers = DataFrame(screenname_Gov_Influentials,columns=['Screen Name of NGovernment Influentials'])
df_followerscount_govInfluencers = DataFrame(no_ofFollowers_Gov_Influentials,columns=['No Of Followers of Government Influentials'])
df_likescount_govInfluencers = DataFrame(no_ofLikes_Gov_Influentials,columns=['No Of Likes of Government Influentials'])
df_followingcount_govInfluencers = DataFrame(no_ofFollowing_Gov_Influentials,columns=['No Of Following for Government Influentials'])
df_tweetscount_govInfluencers = DataFrame(no_ofTweetsCount_Gov_Influentials,columns=['No Of Tweets Shared for Government Influentials'])

concatenated_list = pd.concat([df_scrrenname_govInfluencers, df_followerscount_govInfluencers,df_likescount_govInfluencers, df_followingcount_govInfluencers, df_tweetscount_govInfluencers], axis=1)
concatenated_list

,Screen Name of NGovernment Influentials,No Of Followers of Government Influentials,No Of Likes of Government Influentials,No Of Following for Government Influentials,No Of Tweets Shared for Government Influentials
0,EswatiniGovern1,11315,61,82,1674
1,MalawiGovt,39284,915,26,4030
2,hagegeingob,192408,268,55,1087
3,FinanceSC,126,12,224,125
4,PresidencyZA,1598912,63,14,18881
5,mohzambia,7157,163,95,838
6,edmnangagwa,546288,65,116,628
7,MinSantedj,2932,587,127,1064
8,hawelti,66191,732,435,4717
9,StateHouseKenya,1103712,61,214,9046


##### To get number of retweets, popularity scores, reach scores and relevance scores for each user that is a Government Official

In [65]:
no_ofRetweets_Gov_Influentials = []     
popularityscores_Gov_Influentials = []
reachscores_Gov_Influentials = []
noOfMentions_Gov_Influentials = []

#get number of retweets
for x in range(0, len(gov_officials_twitter_handles)):
    name = gov_officials_twitter_handles[x]
    try:
      Retweet_count = 0
      for status in tweepy.Cursor(api.user_timeline,id=name).items():
        Retweet_count += status.retweet_count
      no_ofRetweets_Gov_Influentials.append(Retweet_count)
    except tweepy.TweepError as e:
        continue

#get mentions for each user

for x in range(0, len(gov_officials_twitter_handles)):
    name = gov_officials_twitter_handles[x]
    try:
        results = api.get_user(id=name)
        mentions = ", ".join([mention[name] for mention in status['entities']['user_mentions']])
        noOfMentions_Gov_Influentials.append(mentions)
    except tweepy.TweepError as e:
        continue

#get popularity scores for each user
for i in range(0, len(no_ofRetweets_Gov_Influentials)):
    popularityscores_Gov_Influentials = no_ofRetweets_Gov_Influentials[i] + no_ofLikes_Gov_Influentials[i]
    popularityscores_Gov_Influentials.append(popularity_score_gov)

#get reach scores for each user
for i in range(0, len(no_ofFollowers_Gov_Influentials)):
    reachscore_gov = no_ofFollowers_Gov_Influentials[i] - no_ofFollowing_Gov_Influentials[i]
    reachscores_Gov_Influentials.append(reachscore_gov)

from pandas import DataFrame
df_retweetcount_Gov_Influencers = DataFrame (no_ofRetweets_Gov_Influentials,columns=['No Of Retweets of Non-Government Influentials'])
df_popularityscores_Gov_Influencers = DataFrame (popularityscores_Gov_Influentials,columns=['Popularity Scores for Non-Government Officials'])
df_reachscores_Gov_Influencers = DataFrame (reachscores_Gov_Influentials,columns=['Reach Scores for Non-Government Influencers'])
df_mentionscores_Gov_Influencers = DataFrame (noOfMentions_Gov_Influentials,columns=['Number Of Mentions for Non-Government Influencers'])

concatenated_list = pd.concat([df_scrrenname_nongovInfluencers, df_retweetcount_Gov_Influencers,df_popularityscores_Gov_Influencers,df_reachscores_Gov_Influencers,
                               df_mentionscores_Gov_Influencers], axis=1)
concatenated_list

TypeError: ignored

##### Extract the top 5 unique hashtags these influencers and government officials used in their top 10 retweets. Group them based on their top 5 hashtags similarities. 

In [60]:
combined_list = []
combined_list = user_twitter_handle + gov_officials_twitter_handles

#get number of retweets
hashtags = []

for x in range(0, len(combined_list)):
  name = combined_list[x]
  firstTweet = api.user_timeline(id=name)
  print(firstTweet.id)

AttributeError: ignored

Make a bar plot to show the fraction of influencers and top government officials by 
hashtag


In [ ]:
import matplotlib as plt
fig, ax = plt.subplots()
g.value_counts().unstack(0).plot(ax=ax, kind='bar', rot=0)
ax.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., fontsize=14)
ax.set_title('Fraction of influencers and gov officials by hashtag', fontsize=18)
ax.set_xlabel('Account Type', fontsize=15)
ax.set_ylabel('Hashtag Count',fontsize=15)

[]